In [3]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from __future__ import division

In [4]:
class LRPI:
    def __init__(self, normalize=False, n_jobs=1, t_value = 2.13144955):
        self.normalize = normalize
        self.n_jobs = n_jobs
        self.LR = linear_model.LinearRegression(normalize=self.normalize, n_jobs= self.n_jobs)
        self.t_value = t_value
        
    def fit(self, X_train, y_train):
        self.X_train = pd.DataFrame(X_train.values)
        self.y_train = pd.DataFrame(y_train.values)
        
        self.LR.fit(self.X_train, self.y_train)
        X_train_fit = self.LR.predict(self.X_train)
        self.MSE = np.power(self.y_train.subtract(X_train_fit), 2).sum(axis=0) / (self.X_train.shape[0] - self.X_train.shape[1] - 1)
        self.X_train.loc[:, 'const_one'] = 1
        self.XTX_inv = np.linalg.inv(np.dot(np.transpose(self.X_train.values) , self.X_train.values))
        
    def predict(self, X_test):
        self.X_test = pd.DataFrame(X_test.values)
        self.pred = self.LR.predict(self.X_test)
        self.X_test.loc[: , 'const_one'] =1
        SE = [np.dot(np.transpose(self.X_test.values[i]) , np.dot(self.XTX_inv, self.X_test.values[i]) ) for i in range(len(self.X_test)) ]
        results = pd.DataFrame(self.pred , columns=['Pred'])
        
        results.loc[:,"lower"] = results['Pred'].subtract((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        results.loc[:,"upper"] = results['Pred'].add((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        
        return results

In [5]:
data = pd.DataFrame(datasets.load_diabetes().data)
target = pd.DataFrame(datasets.load_diabetes().target)

In [6]:
X_train = data.iloc[:-30 ]
y_train = target.iloc[: -30 ]

X_test = data.iloc[-30: ]
y_test = target.iloc[-30: ]

In [8]:
model = LRPI()
model.fit(X_train, y_train)

In [11]:
results = model.predict(X_test)
results.head(10)
print(model.XTX_inv.shape)
print(X_train.shape)

(11, 11)
(412, 10)


In [13]:
np.dot(np.transpose(model.X_train.values) , model.X_train.values)

array([[  9.43201324e-01,   1.45962354e-01,   1.75504197e-01,
          3.03796429e-01,   2.48383317e-01,   2.01398760e-01,
         -4.65022818e-02,   1.74747150e-01,   2.51031548e-01,
          2.74167894e-01,   9.64184188e-02],
       [  1.45962354e-01,   9.33306544e-01,   7.75393842e-02,
          2.10198679e-01,   3.22494715e-02,   1.34628559e-01,
         -3.43981595e-01,   3.05634795e-01,   1.23371545e-01,
          1.98230482e-01,   1.95388032e-01],
       [  1.75504197e-01,   7.75393842e-02,   9.31669973e-01,
          3.54577426e-01,   2.42588027e-01,   2.51700261e-01,
         -3.38508008e-01,   3.99091822e-01,   4.19056369e-01,
          3.56574201e-01,   4.71408665e-02],
       [  3.03796429e-01,   2.10198679e-01,   3.54577426e-01,
          9.29429305e-01,   2.34985087e-01,   1.75159398e-01,
         -1.49237106e-01,   2.37615265e-01,   3.66648810e-01,
          3.62065728e-01,   1.86185047e-01],
       [  2.48383317e-01,   3.22494715e-02,   2.42588027e-01,
          2.34

In [15]:
np.linalg.inv(np.dot(np.transpose(model.X_train.values) , model.X_train.values))

array([[  1.28101416e+00,  -1.29240162e-01,  -1.95213355e-02,
         -2.50014031e-01,   1.36636050e-01,  -2.92813534e-01,
         -1.85390915e-01,   7.43963507e-02,  -2.29651518e-01,
         -1.78354851e-01,  -5.09673755e-04],
       [ -1.29240162e-01,   1.36287459e+00,   2.08653700e-01,
         -2.63140239e-01,   4.08252398e-01,  -2.89247279e-01,
          2.41015287e-01,  -2.96028782e-01,   8.06023146e-02,
         -1.24350710e-01,  -6.14754243e-04],
       [ -1.95213355e-02,   2.08653700e-01,   1.60466297e+00,
         -3.60828247e-01,   7.84686569e-01,  -8.17363510e-01,
          1.01975387e-01,  -1.77521064e-02,  -5.86700482e-01,
         -2.15872396e-01,  -2.80189180e-04],
       [ -2.50014031e-01,  -2.63140239e-01,  -3.60828247e-01,
          1.54099512e+00,  -3.30702420e-01,   1.61240514e-01,
          1.17849684e-01,   3.13946910e-01,  -2.40616305e-01,
         -2.54775123e-01,  -6.23223271e-04],
       [  1.36636050e-01,   4.08252398e-01,   7.84686569e-01,
         -3.30